In [1]:
import csv
import random
import math
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


def loadCsv(filename):
	dataset = pd.read_csv(filename)
	dataset
	unique_activities = dataset.Activity.unique()
	print("Activities {0} Number of unique activities: {1} ".format(unique_activities,len(unique_activities)))
	replacer = {}
	for i, activity in enumerate(unique_activities):
		replacer[activity] = i
	dataset.Activity = dataset.Activity.replace(replacer)
	test_data = dataset.iloc[1:].as_matrix()
	datasetfinal = list(test_data)
	for i in range(len(datasetfinal)):
		datasetfinal[i] = [float(x) for x in datasetfinal[i]]
	return datasetfinal

def splitDataset(dataset, splitRatio):
	trainSize = int(len(dataset) * splitRatio)
	trainSet = []
	copy = list(dataset)
	while len(trainSet) < trainSize:
		index = random.randrange(len(copy))
		trainSet.append(copy.pop(index))
	return [trainSet, copy]

def separateByClass(dataset):
	separated = {}
	for i in range(len(dataset)):
		vector = dataset[i]
		if (vector[-1] not in separated):
			separated[vector[-1]] = []
		separated[vector[-1]].append(vector)
	return separated

def mean(numbers):
	return sum(numbers)/float(len(numbers))

def stdev(numbers):
	avg = mean(numbers)
	variance = sum([pow(x-avg,2) for x in numbers])/float(len(numbers)-1)
	return math.sqrt(variance)

def summarize(dataset):
	summaries = [(mean(attribute), stdev(attribute)) for attribute in zip(*dataset)]
	del summaries[-1]
	return summaries

def summarizeByClass(dataset):
	separated = separateByClass(dataset)
	summaries = {}
	for classValue, instances in separated.items():
		summaries[classValue] = summarize(instances)
	return summaries

def calculateProbability(x, mean, stdev):
	try:
		exponent = math.exp(-(math.pow(x-mean,2)/(2*math.pow(stdev,2))))
	except ZeroDivisionError:
		exponent = 1
	return (1 / (math.sqrt(2*math.pi) * stdev)) * exponent

def calculateClassProbabilities(summaries, inputVector):
	probabilities = {}
	for classValue, classSummaries in summaries.items():
		probabilities[classValue] = 1
		for i in range(len(classSummaries)):
			mean, stdev = classSummaries[i]
			x = inputVector[i]
			probabilities[classValue] *= calculateProbability(x, mean, stdev)
	return probabilities
			
def predict(summaries, inputVector):
	probabilities = calculateClassProbabilities(summaries, inputVector)
	bestLabel, bestProb = None, -1
	for classValue, probability in probabilities.items():
		if bestLabel is None or probability > bestProb:
			bestProb = probability
			bestLabel = classValue
	return bestLabel

def getPredictions(summaries, testSet):
	predictions = []
	for i in range(len(testSet)):
		result = predict(summaries, testSet[i])
		predictions.append(result)
	return predictions

def getAccuracy(testSet, predictions):
	correct = 0
	for i in range(len(testSet)):
		if testSet[i][-1] == predictions[i]:
			correct += 1
	return (correct/float(len(testSet))) * 100.0

def main():
	print('-----------------')
	print('---Naive Bayes---')
	print('-----------------')
	filename = 'C:/Users/Sebastian/Documents/BIG DATA/Labs/Lab7/DataTest.csv'
	splitRatio = 0.67
	datasetfinal = loadCsv(filename)
#Split The DataSet in trainSet or TestSet
	trainingSet, testSet = splitDataset(datasetfinal, splitRatio)
	summaries = summarizeByClass(trainingSet)
	# test model
	predictions = getPredictions(summaries, testSet)
	accuracy = getAccuracy(testSet, predictions)
	print('Accuracy: {0}%'.format(accuracy))

In [2]:
main()

-----------------
---Naive Bayes---
-----------------
Activities ['WALKING' 'UPSTAIRS' 'DOWNSTAIRS' 'FALLING' 'LAYING' 'SITTING' 'STANDING'] Number of unique activities: 7 


C:\Users\Sebastian\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


Accuracy: 41.24021673690548%
